# ABS Quarterly Producer Prices Index  6427

## Python set-up

In [1]:
# system imports
import sys
import pathlib  
import re

# analytic imports
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# local imports
from plotting import (
    finalise_plot,
    set_chart_dir,
    recalibrate_series,
    clear_chart_dir,
    plot_growth_finalise,
    calc_growth,
)    
from abs_data_capture import (
    get_fs_constants,
    get_ABS_meta_and_data,
    get_plot_constants,
    find_id,
    clear_cache,
)

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# plotting stuff
plt.style.use('fivethirtyeight')

In [2]:
catalogue_id = '6427'
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)

## Get data from ABS

In [3]:
# do the data capture and extraction to a DataFrame
abs_dict = get_ABS_meta_and_data(catalogue_id)
if abs_dict is None:
    sys.exit(-1)

Found URL for a ZIP file on ABS web page
Retrieving zip-file from cache ...
Extracting DataFrames from the zip-file ...


## Plot

### Set-up some constants we will use across all plots

In [4]:
meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
#meta

In [5]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)

### Headline PPI for Australia

In [6]:
quarterly_phrase = 'Percentage change from previous quarter'
annual_phrase = 'Percentage change from corresponding quarter of previous year'

desc_col = 'Data Item Description'
table_col = 'Table'

final = 'Final ;  Total (Source)'

In [7]:
get = {
    (final, '1'): 'Final Demand',
    ('Coal mining', '11'): 'Coal mining',
    ('Manufacturing division', '12'): 'Manufacturing',
    #('Building construction Australia', '17'): 'Building construction Australia',
    #('Road freight transport ', '21'): ' Road freight transport ',
    
}

for key, label in get.items():
    select_phrase, table = key
    data = abs_dict[table]
    identifiers = []
    for phrase in annual_phrase, quarterly_phrase:
        terms = {
            table: table_col,
            select_phrase: desc_col, 
            phrase: desc_col, 
        }
        ident, units = find_id(meta, terms, verbose=False)
        identifiers.append(ident)

    # plot
    title = f'Growth: {label} PPI'
    for plot_from, plot_tag in zip(plot_times, plot_tags):
        p_plot_from = None if plot_from is None else pd.Period(plot_from, freq=data.index.freq)
        plot_growth_finalise(
            data[identifiers[0]], data[identifiers[1]], 
            title=title, 
            from_=p_plot_from, 
            tag=plot_tag, 
            rfooter=f'{source} {table}',
            lfooter=f'Australia: Producer Price Index. Original series',
            #show=True
        )
    

### Lesser PPIs - no ABS pre-calculated percentages

In [8]:
get = {
    ('30 Building construction Australia ;', '17'): 'Building construction Australia',
    ('Road freight transport ', '21'): 'Road freight transport',
    ('Employment services', '25'): 'Employment services',
    
}

for key, label in get.items():
    select_phrase, table = key
    data = abs_dict[table]
    terms = {
            table: table_col,
            select_phrase: desc_col, 
            'Index Number': desc_col, 
    }
    ident, units = find_id(meta, terms, verbose=False)
    growth = calc_growth(abs_dict[table][ident])
    
    # plot
    title = f'Growth: {label} PPI'
    for plot_from, plot_tag in zip(plot_times, plot_tags):
        p_plot_from = None if plot_from is None else pd.Period(plot_from, freq=data.index.freq)
        plot_growth_finalise(
            *growth, 
            title=title, 
            from_=p_plot_from, 
            tag=plot_tag, 
            rfooter=f'{source} {table}',
            lfooter=f'Australia: Producer Price Index. Original series',
            #show=True
        )


## Finished

In [9]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Feb 22 2023 08:41:51

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.10.0

re        : 2.2.1
numpy     : 1.24.2
matplotlib: 3.7.0
sys       : 3.11.0 | packaged by conda-forge | (main, Jan 15 2023, 05:44:48) [Clang 14.0.6 ]
pandas    : 1.5.3

Watermark: 2.3.1



In [10]:
print('Finished')

Finished
